In [1]:
import theano

/home/wnixalo/miniconda3/envs/FAI2/lib/python2.7/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX 870M (0000:01:00.0)


In [2]:
import os, sys
sys.path.insert(1, os.path.join('utils'))

from __future__ import print_function, division

path = 'data/statefarm/'
import utils; reload(utils)
from utils import *

Using Theano backend.


In [3]:
batch_size=4

In [4]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i, l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx + 1]
conv_model = Sequential(conv_layers)

In [5]:
test_batches = get_batches(path + 'test', batch_size=batch_size, shuffle=False)

Found 79726 images belonging to 1 classes.


In [6]:
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

MemoryError: 